In [2]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


import keras
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LayerNormalization
from keras.models import Sequential

import yellowbrick
from yellowbrick.datasets import load_concrete

In [3]:
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("sklearn:", sklearn.__version__)
print("keras:", keras.__version__)
print("yellowbrick:", yellowbrick.__version__)

numpy: 1.21.5
pandas: 1.4.1
sklearn: 1.0.2
keras: 2.7.0
yellowbrick: 1.4


In [4]:
X, y = load_concrete()

In [5]:
X

,cement,slag,ash,water,splast,coarse,fine,age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360
...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28


In [6]:
y

0       79.986111
1       61.887366
2       40.269535
3       41.052780
4       44.296075
          ...    
1025    44.284354
1026    31.178794
1027    23.696601
1028    32.768036
1029    32.401235
Name: strength, Length: 1030, dtype: float64

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=None, shuffle=True)

In [8]:
def K_Regre(): 
    keras.backend.clear_session()
#   neural network architecture: start  
    model = Sequential() 
    model.add(Dense(15, activation='relu')) 
    model.add(BatchNormalization()) 
#    model.add(LayerNormalization()) 
    model.add(Dense(1)) 
#   neural network architecture: end   
    model.compile(loss='mean_squared_error', optimizer='adam') 
    return model

In [9]:
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=20)


In [15]:
K_R = KerasRegressor(K_Regre,batch_size=64, epochs=2000, callbacks=[es],validation_data=(X_test, y_test), verbose=1) 
K_R._estimator_type = 'regressor'

<ipython-input-15-e2c7326b8c95>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  K_R = KerasRegressor(K_Regre,batch_size=64, epochs=2000, callbacks=[es],validation_data=(X_test, y_test), verbose=1)


In [16]:
level_0 = [ 
          ('ELNE', ElasticNet(alpha=0.01, l1_ratio=0.5)), 
          ('KERR', K_R), 
          ]

In [17]:
level_1 = LinearRegression()

In [18]:
model = StackingRegressor(level_0, final_estimator=level_1)

In [19]:
model.fit(X_train, y_train)

Epoch 1/2000
11/11 [==============================] - 1s 31ms/step - loss: 1595.4772 - val_loss: 2020.3481
Epoch 2/2000
11/11 [==============================] - 0s 9ms/step - loss: 1589.6898 - val_loss: 1849.2654
Epoch 3/2000
11/11 [==============================] - 0s 12ms/step - loss: 1583.2938 - val_loss: 1774.8831
Epoch 4/2000
11/11 [==============================] - 0s 9ms/step - loss: 1576.5785 - val_loss: 1730.1853
Epoch 5/2000
11/11 [==============================] - 0s 9ms/step - loss: 1569.4993 - val_loss: 1702.2384
Epoch 6/2000
11/11 [==============================] - 0s 9ms/step - loss: 1561.2898 - val_loss: 1676.2925
Epoch 7/2000
11/11 [==============================] - 0s 10ms/step - loss: 1552.8201 - val_loss: 1647.3811
Epoch 8/2000
11/11 [==============================] - 0s 11ms/step - loss: 1543.7280 - val_loss: 1622.1842
Epoch 9/2000
11/11 [==============================] - 0s 10ms/step - loss: 1533.0184 - val_loss: 1609.1033
Epoch 10/2000
11/11 [====================

11/11 [==============================] - 0s 9ms/step - loss: 293.9218 - val_loss: 264.1549
Epoch 79/2000
11/11 [==============================] - 0s 8ms/step - loss: 284.1435 - val_loss: 219.1869
Epoch 80/2000
11/11 [==============================] - 0s 8ms/step - loss: 267.9100 - val_loss: 230.9834
Epoch 81/2000
11/11 [==============================] - 0s 8ms/step - loss: 257.6327 - val_loss: 230.4518
Epoch 82/2000
11/11 [==============================] - 0s 8ms/step - loss: 246.4520 - val_loss: 255.5000
Epoch 83/2000
11/11 [==============================] - 0s 9ms/step - loss: 231.2897 - val_loss: 222.1291
Epoch 84/2000
11/11 [==============================] - 0s 9ms/step - loss: 223.7023 - val_loss: 228.1414
Epoch 85/2000
11/11 [==============================] - 0s 11ms/step - loss: 214.2912 - val_loss: 210.9411
Epoch 86/2000
11/11 [==============================] - 0s 9ms/step - loss: 204.2153 - val_loss: 228.7777
Epoch 87/2000
11/11 [==============================] - 0s 12ms/step 

11/11 [==============================] - 0s 11ms/step - loss: 38.4373 - val_loss: 49.3048
Epoch 157/2000
11/11 [==============================] - 0s 9ms/step - loss: 42.1831 - val_loss: 45.7371
Epoch 158/2000
11/11 [==============================] - 0s 13ms/step - loss: 45.3581 - val_loss: 44.9804
Epoch 159/2000
11/11 [==============================] - 0s 9ms/step - loss: 40.7258 - val_loss: 48.3382
Epoch 160/2000
11/11 [==============================] - 0s 10ms/step - loss: 43.3024 - val_loss: 43.9138
Epoch 161/2000
11/11 [==============================] - 0s 9ms/step - loss: 40.7327 - val_loss: 46.1046
Epoch 162/2000
11/11 [==============================] - 0s 8ms/step - loss: 41.2503 - val_loss: 44.4852
Epoch 163/2000
11/11 [==============================] - 0s 9ms/step - loss: 43.9664 - val_loss: 43.7740
Epoch 164/2000
11/11 [==============================] - 0s 12ms/step - loss: 40.3871 - val_loss: 44.0182
Epoch 165/2000
11/11 [==============================] - 0s 9ms/step - loss:

9/9 [==============================] - 0s 12ms/step - loss: 1637.8062 - val_loss: 837.4780
Epoch 4/2000
9/9 [==============================] - 0s 10ms/step - loss: 1630.4908 - val_loss: 937.6667
Epoch 5/2000
9/9 [==============================] - 0s 12ms/step - loss: 1623.0229 - val_loss: 1013.8231
Epoch 6/2000
9/9 [==============================] - 0s 13ms/step - loss: 1615.0732 - val_loss: 1076.2643
Epoch 7/2000
9/9 [==============================] - 0s 12ms/step - loss: 1608.4518 - val_loss: 1120.3395
Epoch 8/2000
9/9 [==============================] - 0s 10ms/step - loss: 1599.8829 - val_loss: 1143.4652
Epoch 9/2000
9/9 [==============================] - 0s 12ms/step - loss: 1591.3433 - val_loss: 1165.4719
Epoch 10/2000
9/9 [==============================] - 0s 12ms/step - loss: 1582.5640 - val_loss: 1185.9998
Epoch 11/2000
9/9 [==============================] - 0s 11ms/step - loss: 1573.8463 - val_loss: 1212.7880
Epoch 12/2000
9/9 [==============================] - 0s 10ms/step - 

9/9 [==============================] - 0s 11ms/step - loss: 1116.6787 - val_loss: 829.5233
Epoch 38/2000
9/9 [==============================] - 0s 12ms/step - loss: 1102.1531 - val_loss: 764.7782
Epoch 39/2000
9/9 [==============================] - 0s 10ms/step - loss: 1080.9819 - val_loss: 716.4703
Epoch 40/2000
9/9 [==============================] - 0s 12ms/step - loss: 1066.6370 - val_loss: 656.6973
Epoch 41/2000
9/9 [==============================] - 0s 13ms/step - loss: 1046.4663 - val_loss: 613.7878
Epoch 42/2000
9/9 [==============================] - 0s 11ms/step - loss: 1029.9221 - val_loss: 571.0283
Epoch 43/2000
9/9 [==============================] - 0s 11ms/step - loss: 1012.5709 - val_loss: 534.9085
Epoch 44/2000
9/9 [==============================] - 0s 12ms/step - loss: 997.8595 - val_loss: 482.6661
Epoch 45/2000
9/9 [==============================] - 0s 11ms/step - loss: 976.4050 - val_loss: 440.4130
Epoch 46/2000
9/9 [==============================] - 0s 13ms/step - los

9/9 [==============================] - 0s 11ms/step - loss: 1318.2970 - val_loss: 1131.3398
Epoch 29/2000
9/9 [==============================] - 0s 13ms/step - loss: 1303.5270 - val_loss: 1101.1278
Epoch 30/2000
9/9 [==============================] - 0s 11ms/step - loss: 1284.7825 - val_loss: 1076.9470
Epoch 31/2000
9/9 [==============================] - 0s 11ms/step - loss: 1271.3267 - val_loss: 1045.9153
Epoch 32/2000
9/9 [==============================] - 0s 12ms/step - loss: 1253.7150 - val_loss: 1041.3641
Epoch 33/2000
9/9 [==============================] - 0s 11ms/step - loss: 1237.1011 - val_loss: 1028.3119
Epoch 34/2000
9/9 [==============================] - 0s 12ms/step - loss: 1219.6438 - val_loss: 1001.9919
Epoch 35/2000
9/9 [==============================] - 0s 11ms/step - loss: 1203.4083 - val_loss: 983.5321
Epoch 36/2000
9/9 [==============================] - 0s 13ms/step - loss: 1186.0795 - val_loss: 958.5378
Epoch 37/2000
9/9 [==============================] - 0s 11ms/s

9/9 [==============================] - 0s 11ms/step - loss: 200.0712 - val_loss: 105.1178
Epoch 107/2000
9/9 [==============================] - 0s 11ms/step - loss: 190.0958 - val_loss: 96.1214
Epoch 108/2000
9/9 [==============================] - 0s 13ms/step - loss: 186.5228 - val_loss: 97.0166
Epoch 109/2000
9/9 [==============================] - 0s 12ms/step - loss: 179.5111 - val_loss: 88.9700
Epoch 110/2000
9/9 [==============================] - 0s 13ms/step - loss: 174.7970 - val_loss: 97.4096
Epoch 111/2000
9/9 [==============================] - 0s 10ms/step - loss: 163.1622 - val_loss: 83.4545
Epoch 112/2000
9/9 [==============================] - 0s 16ms/step - loss: 160.7828 - val_loss: 76.9211
Epoch 113/2000
9/9 [==============================] - 0s 13ms/step - loss: 155.3980 - val_loss: 83.2763
Epoch 114/2000
9/9 [==============================] - 0s 11ms/step - loss: 150.4912 - val_loss: 75.6032
Epoch 115/2000
9/9 [==============================] - 0s 13ms/step - loss: 143

StackingRegressor(estimators=[('ELNE', ElasticNet(alpha=0.01)),
                              ('KERR',
                               <keras.wrappers.scikit_learn.KerasRegressor object at 0x7fb1645e2f40>)],
                  final_estimator=LinearRegression())

In [20]:
model.predict(X_test[:2])

1/1 [==============================] - 0s 110ms/step


array([24.19403002, 45.78092171])

In [21]:
model.predict(X_test[:1])

1/1 [==============================] - 0s 35ms/step


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [22]:
model_K = K_R

In [23]:
model_K.fit(X_train, y_train)

Epoch 1/2000
11/11 [==============================] - 1s 26ms/step - loss: 1595.5521 - val_loss: 1739.2104
Epoch 2/2000
11/11 [==============================] - 0s 13ms/step - loss: 1587.6556 - val_loss: 1714.5410
Epoch 3/2000
11/11 [==============================] - 0s 10ms/step - loss: 1579.7306 - val_loss: 1709.0000
Epoch 4/2000
11/11 [==============================] - 0s 9ms/step - loss: 1571.7888 - val_loss: 1712.1527
Epoch 5/2000
11/11 [==============================] - 0s 15ms/step - loss: 1562.8529 - val_loss: 1701.8569
Epoch 6/2000
11/11 [==============================] - 0s 10ms/step - loss: 1553.4917 - val_loss: 1691.4114
Epoch 7/2000
11/11 [==============================] - 0s 9ms/step - loss: 1543.2665 - val_loss: 1686.1919
Epoch 8/2000
11/11 [==============================] - 0s 10ms/step - loss: 1532.7073 - val_loss: 1665.6783
Epoch 9/2000
11/11 [==============================] - 0s 13ms/step - loss: 1520.7286 - val_loss: 1620.1686
Epoch 10/2000
11/11 [==================

Epoch 78/2000
11/11 [==============================] - 0s 14ms/step - loss: 271.9914 - val_loss: 169.9856
Epoch 79/2000
11/11 [==============================] - 0s 11ms/step - loss: 262.5677 - val_loss: 155.8147
Epoch 80/2000
11/11 [==============================] - 0s 11ms/step - loss: 251.9163 - val_loss: 171.4501
Epoch 81/2000
11/11 [==============================] - 0s 13ms/step - loss: 245.2926 - val_loss: 130.3772
Epoch 82/2000
11/11 [==============================] - 0s 13ms/step - loss: 233.5275 - val_loss: 147.8282
Epoch 83/2000
11/11 [==============================] - 0s 14ms/step - loss: 223.2791 - val_loss: 141.5839
Epoch 84/2000
11/11 [==============================] - 0s 13ms/step - loss: 214.4541 - val_loss: 138.8086
Epoch 85/2000
11/11 [==============================] - 0s 15ms/step - loss: 201.9308 - val_loss: 138.7660
Epoch 86/2000
11/11 [==============================] - 0s 10ms/step - loss: 194.0577 - val_loss: 149.7583
Epoch 87/2000
11/11 [=========================

Epoch 156/2000
11/11 [==============================] - 0s 10ms/step - loss: 42.1537 - val_loss: 50.4841
Epoch 157/2000
11/11 [==============================] - 0s 10ms/step - loss: 43.9226 - val_loss: 43.6672
Epoch 158/2000
11/11 [==============================] - 0s 12ms/step - loss: 43.4223 - val_loss: 44.9582
Epoch 159/2000
11/11 [==============================] - 0s 12ms/step - loss: 42.8841 - val_loss: 45.0228
Epoch 160/2000
11/11 [==============================] - 0s 10ms/step - loss: 45.8298 - val_loss: 44.4025
Epoch 161/2000
11/11 [==============================] - 0s 12ms/step - loss: 43.7008 - val_loss: 47.6002
Epoch 162/2000
11/11 [==============================] - 0s 14ms/step - loss: 42.3005 - val_loss: 45.0934
Epoch 163/2000
11/11 [==============================] - 0s 10ms/step - loss: 45.4406 - val_loss: 44.3587
Epoch 164/2000
11/11 [==============================] - 0s 13ms/step - loss: 43.0125 - val_loss: 45.5621
Epoch 165/2000
11/11 [==============================] -

In [24]:
model_K.predict(X_test[:1])

1/1 [==============================] - 0s 100ms/step


array(22.85553, dtype=float32)

In [25]:
model_K.predict(X_test[:2])

1/1 [==============================] - 0s 33ms/step


array([22.855536, 41.029778], dtype=float32)